In [9]:
%load_ext autoreload
%autoreload 2
# %flow mode reactive

from importlib import reload
from pathlib import Path
import time

from dotmap import DotMap
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import seaborn as sns
from tqdm import tqdm

import aeon
from aeon.analysis.utils import visits, distancetravelled
from aeon.io import reader
from aeon.io.device import Device, register
from aeon.schema import core, foraging, social
from aeon.schema.schemas import exp02, social01, social02
import datajoint as dj

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Prettify pandas output display.

pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 50)

## Create VirtualModule to access `aeon_test_analysis` schema
Currently, the analysis is on `aeon_test_`, will move to `aeon_` soon (once ready for production)

In [ ]:
analysis_vm = dj.create_virtual_module('aeon_test_analysis', 'aeon_test_analysis')

## Browse Block and BlockAnalysis

In [ ]:
# View ERD around analysis

dj.Diagram(analysis_vm) - 1

In [ ]:
# View Block table

analysis_vm.Block()

In [ ]:
# Fetch BlockAnalysis table

blocks = analysis_vm.BlockAnalysis().fetch(format="frame")
blocks

In [ ]:
# Spec block(s) of interest

# block_key = {
#     "experiment_name": "social0.1-aeon3",
#     "block_start": "2023-12-05 11:06:01.001984",
# } 
block_start = "block_start LIKE '2023-12-03%'"
experiment_name = "experiment_name LIKE 'social0.1-aeon3'"

In [ ]:
# Given above keys, restrict BlockAnalysis table, and select info of first of returned blocks

block_df = (analysis_vm.BlockAnalysis & block_start & experiment_name).fetch(format="frame")
display(block_df)
block_key = {
    "experiment_name": block_df.index[0][0],
    "block_start": block_df.index[0][1]
}
print(block_key)

In [ ]:
# Return BlockAnalysis' part tables

analysis_vm.BlockAnalysis.parts()

In [ ]:
# For each part table, display and fetch info for the block of interest

display(analysis_vm.BlockAnalysis.Patch & block_key)
display(analysis_vm.BlockAnalysis.Subject & block_key)
block_patch_data = (analysis_vm.BlockAnalysis.Patch & block_key).fetch(as_dict=True)
block_subject_data = (analysis_vm.BlockAnalysis.Subject & block_key).fetch(as_dict=True)


## Corral some data

In [3]:
"""Create a blocks df."""

exp_start = pd.Timestamp("2024-01-31 00:00:00")
exp_end = pd.Timestamp("2024-02-17 00:00:00")
# roots = [
#     Path("/ceph/aeon/aeon/data/raw/AEON3/social0.2"),
#     Path("/ceph/aeon/aeon/data/raw/AEON4/social0.2"),
# ]
roots = [
    Path(r"Z:\aeon\data\raw\AEON3\social0.2"),
    Path(r"Z:\aeon\data\raw\AEON4\social0.2"),
]
social02.CameraTop.Pose._model_root = Path(r"Z:\aeon\data\processed")
arenas = ["AEON3", "AEON4"]
patches = ["Patch1", "Patch2", "Patch3"]
patch_locs = pd.DataFrame(index=arenas, columns=patches)
blocks_df = pd.DataFrame()
block_ts_tol = pd.Timedelta("2s")  # Tolerance for block start and end times
good_block_pel_ct = 4  # Min pellets for good block


for root in roots:
    # Pull out info from metadata
    for arena in arenas:
        if arena in str(root):
            break
    metadata = aeon.load(root, social02.Metadata, exp_start, exp_end).iloc[0].metadata
    patch_locs.loc[arena, patches] = (
        (metadata.Devices.Patch1Rfid.Location.X, metadata.Devices.Patch1Rfid.Location.Y),
        (metadata.Devices.Patch2Rfid.Location.X, metadata.Devices.Patch2Rfid.Location.Y),
        (metadata.Devices.Patch3Rfid.Location.X, metadata.Devices.Patch3Rfid.Location.Y)
    )
    block_info = aeon.load(root, social02.Environment.BlockState, exp_start, exp_end)

    # Block end if pellet_ct == 0 and preceding pellet_ct > 0
    # OR pellet_ct == 0, preceding pellet_ct == 0, and preceding pellet_ct time diff > 1s
    possible_block_end_indxs = np.where(block_info.pellet_ct == 0)[0]
    drop_indxs = []

    for i in possible_block_end_indxs:
        if block_info.pellet_ct[i - 1] > 0:
            continue
        elif block_info.pellet_ct[i - 1] == 0 and block_info.index[i] - block_info.index[i - 1] > pd.Timedelta("1s"):
            continue
        else:  # drop i from `possible_block_ends`
            drop_indxs.append(i)

    block_end_indxs = np.setdiff1d(possible_block_end_indxs, drop_indxs)
    # Start from first complete block to last complete block
    block_start_indxs = block_end_indxs[0:-1] 
    block_end_indxs = block_end_indxs[1:]
    block_start_times = block_info.index[block_start_indxs] - block_ts_tol
    block_end_times = block_info.index[block_end_indxs] + block_ts_tol
    sleap_model_dir = (
        Path(r"Z:/aeon/data/processed/test-node1/4310907/2024-01-12T19-00-00/topdown-multianimal-id-133")
        if "AEON3" in str(root) else 
        Path(r"Z:/aeon/data/processed/test-node1/4350621/2024-01-22T19-00-00/topdown-multianimal-id-133")
    )

    # Create a `blocks` df with columns 'start', 'end', and 'root'
    blocks_df = pd.concat(
        [
         blocks_df,
         pd.DataFrame(
             {   
                 "root": [root] * len(block_start_times),
                 "sleap_model_dir": [sleap_model_dir] * len(block_start_times),
                 "start": block_start_times, 
                 "end": block_end_times, 
             }
        )
       ], ignore_index=True
    )

# Add columns to `blocks_df`
new_cols = [
    "block_duration",
    "subjects",  # list of subjects in block
    "patch_info",  # df: index: patch; cols: rate, offset
    "pellet_info",  # df: index: del ts; cols: patch, thresh, id {for each pel del, get last thresh}
    "cum_wheel_dist",  # DotMap: patch: df
]
for col in new_cols:
    blocks_df[col] = None

blocks_df = blocks_df.sort_values(by="start").reset_index(drop=True)
display(blocks_df)

C:\Users\jai\AppData\Local\Temp\ipykernel_60492\3226657415.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if block_info.pellet_ct[i - 1] > 0:
C:\Users\jai\AppData\Local\Temp\ipykernel_60492\3226657415.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif block_info.pellet_ct[i - 1] == 0 and block_info.index[i] - block_info.index[i - 1] > pd.Timedelta("1s"):
C:\Users\jai\ProjectAeon\aeon_mecha\aeon\io\api.py:139: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old

,root,sleap_model_dir,start,end,block_duration,subjects,patch_info,pellet_info,cum_wheel_dist
0,Z:\aeon\data\raw\AEON4\social0.2,Z:\aeon\data\processed\test-node1\4350621\2024...,2024-01-31 12:59:06.005983829,2024-01-31 14:58:11.045983791,None,None,None,None,None
1,Z:\aeon\data\raw\AEON3\social0.2,Z:\aeon\data\processed\test-node1\4310907\2024...,2024-01-31 12:59:14.001984119,2024-01-31 14:45:59.000000000,None,None,None,None,None
2,Z:\aeon\data\raw\AEON3\social0.2,Z:\aeon\data\processed\test-node1\4310907\2024...,2024-01-31 14:45:55.000000000,2024-01-31 16:18:11.001984119,None,None,None,None,None
3,Z:\aeon\data\raw\AEON4\social0.2,Z:\aeon\data\processed\test-node1\4350621\2024...,2024-01-31 14:58:07.045983791,2024-01-31 17:49:26.000000000,None,None,None,None,None
4,Z:\aeon\data\raw\AEON3\social0.2,Z:\aeon\data\processed\test-node1\4310907\2024...,2024-01-31 16:18:07.001984119,2024-01-31 17:56:23.000000000,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...
300,Z:\aeon\data\raw\AEON4\social0.2,Z:\aeon\data\processed\test-node1\4350621\2024...,2024-02-15 04:14:04.005983829,2024-02-15 05:57:04.017983913,None,None,None,None,None
301,Z:\aeon\data\raw\AEON4\social0.2,Z:\aeon\data\processed\test-node1\4350621\2024...,2024-02-15 05:57:00.017983913,2024-02-15 07:10:00.077983856,None,None,None,None,None
302,Z:\aeon\data\raw\AEON4\social0.2,Z:\aeon\data\processed\test-node1\4350621\2024...,2024-02-15 07:09:56.077983856,2024-02-15 09:06:53.001984119,None,None,None,None,None
303,Z:\aeon\data\raw\AEON4\social0.2,Z:\aeon\data\processed\test-node1\4350621\2024...,2024-02-15 09:06:49.001984119,2024-02-15 10:18:32.001984119,None,None,None,None,None


In [4]:
"""Get subject env visits."""

subject_env_visits = {}

for root, arena in zip(roots, arenas):
    subject_visits = aeon.load(root, social02.Environment.SubjectVisits, exp_start, exp_end)
    # Find all rows where:
    #  - 'id' column starts with "*AA"
    #  - 'type' column is either "Enter" or "Exit" or "Remain",
    #  - 'region' column is "Environment"
    subject_visits = subject_visits[
        (subject_visits.id.str.contains("^.*AA"))
        & (subject_visits.type.isin(["Enter", "Exit", "Remain"]))
        & (subject_visits.region == "Environment")
    ]
    subject_env_visits[arena] = subject_visits

C:\Users\jai\ProjectAeon\aeon_mecha\aeon\io\api.py:139: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  data = pd.concat([reader.read(file) for _, file in files])


In [ ]:
subject_env_visits

In [20]:
"""Fill out blocks df."""

skipped_blocks = np.full(len(blocks_df), False)

for i, block in tqdm(enumerate(blocks_df.itertuples()), desc="Block Iteration:"):
    # Compute block duration
    blocks_df.at[block.Index, "block_duration"] = block.end - block.start
    # <s Get subjects within the block:
    # Get all unique subjects that visited the environment over the entire exp;
    # For each subject, see 'type' of visit most recent to start of block;
    # If "Exit", this animal was not in the block.
    for arena in arenas:
        if arena in str(block.root):
            cur_env_visits = subject_env_visits[arena]
            break
    possible_subjects = cur_env_visits.id.unique().tolist()
    subjects = possible_subjects.copy()
    for subject in possible_subjects:
        subj_visit_in_time = np.logical_and(
            cur_env_visits.id == subject, cur_env_visits.index < block.start
        )
        if not np.any(subj_visit_in_time):  # if no subject visits prior to block start, drop it
            subjects.remove(subject)
        else:  # if visits, get most recent visit type before block; if "Exit", drop it
            pre_block_visit = cur_env_visits[subj_visit_in_time].iloc[-1]
            # last_visit = cur_env_visits[cur_env_visits.id == subject].iloc[-1]
            if pre_block_visit.type == "Exit":
                subjects.remove(subject)
    blocks_df.at[block.Index, "subjects"] = subjects
    # /s>
    # <s See if we should continue with analyzing this block
    cum_pel_ct = 0
    for patch in patches:
        r = eval(f"social02.{patch}.DepletionState")
        patch_df = aeon.load(block.root, r, block.start, block.end)
        cum_pel_ct += sum(np.diff(patch_df.index) > pd.Timedelta("1s"))
    if (cum_pel_ct < good_block_pel_ct) or (len(subjects) < 1):
        skipped_blocks[i] = True
        continue
    # /s>
    # <s Get pose-tracking info in order to do subject-specific assignments
    pose_df = aeon.load(block.root, social02.CameraTop.Pose, block.start, block.end)
    pose_df = reader.Pose.class_int2str(pose_df, block.sleap_model_dir)
    if len(subjects) == 1:  # fix mistaken sleap assignments for single-subject blocks
        pose_df["class"] = subjects[0]
    # /s>
    # <s Get per patch data (fill in `patch_info`, `cum_wheel_dist`, `pellet_info` cols of `blocks_df`)
    patch_stats_df = pd.DataFrame(index=patches, columns=["mean", "offset"])  # -> patch_info
    cum_wheel_dist_dm = DotMap()  # -> cum_wheel_dist
    pellets_stats_df = pd.DataFrame(columns=["time", "patch", "threshold", "id"])  # -> pellet_info
    for i, patch in enumerate(patches):
        # <ss Get wheel data
        r = eval(f"social02.{patch}.Encoder")
        wheel_df = aeon.load(block.root, r, block.start, block.end)[::50]
        cum_wheel_dist = -distancetravelled(wheel_df.angle)
        # /ss>
        # <ss Get pellets data
        r = eval(f"social02.{patch}.DepletionState")
        patch_df = aeon.load(block.root, r, block.start, block.end)
        rate, offset = patch_df[["rate", "offset"]].iloc[0]
        patch_stats_df.loc[patch, ["mean", "offset"]] = (1 / rate // 100 * 100, offset)
        patch_df_good_indxs = np.concatenate((np.diff(patch_df.index) > pd.Timedelta("1s"), (True,)))
        patch_df_for_pellets_df = patch_df[patch_df_good_indxs].reset_index()[["time", "threshold"]]
        patch_df_for_pellets_df["patch"] = patch
        patch_df_for_pellets_df["id"] = None
        patch_df_for_pellets_df.dropna(subset=["threshold"], inplace=True)
        # drop 1st val as is from block start
        patch_df_for_pellets_df = patch_df_for_pellets_df.iloc[1:].reset_index(drop=True)
        # /ss>
        # <ss Assign data to subjects
        if len(subjects) == 1:
            cum_wheel_dist_dm[patch] = cum_wheel_dist.to_frame(name=subjects[0])
            patch_df_for_pellets_df["id"] = subjects[0]
        else:
            # <sss Assign id based on which subject was closest to patch at time of event
            # <ssss Get distance-to-patch at each pose data timestep
            patch_xy = np.array(patch_locs[patch][arena]).astype(np.uint32)
            subjects_xy = pose_df[pose_df["part"] == "centroid"][["x", "y"]].values
            dist_to_patch = np.sqrt(np.sum((subjects_xy - patch_xy) ** 2, axis=1))
            dist_to_patch_df = pose_df[["class"]].copy()
            dist_to_patch_df["dist_to_patch"] = dist_to_patch
            # /ssss>
            # <ssss Get distance-to-patch at each wheel ts and pel del ts, organized by subject
            dist_to_patch_wheel_ts_id_df = pd.DataFrame(index=cum_wheel_dist.index, columns=subjects)
            dist_to_patch_pel_ts_id_df = pd.DataFrame(
                index=patch_df_for_pellets_df["time"], columns=subjects
            )
            for subject in subjects:
                # Find closest match between pose_df indices and wheel indices
                dist_to_patch_wheel_ts_subj = pd.merge_asof(
                    left=dist_to_patch_wheel_ts_id_df[subject],
                    right=dist_to_patch_df[dist_to_patch_df["class"] == subject],
                    left_index=True,
                    right_index=True,
                    direction="forward",
                    tolerance=pd.Timedelta("100ms"),
                )
                dist_to_patch_wheel_ts_id_df[subject] = dist_to_patch_wheel_ts_subj["dist_to_patch"]
                # Find closest match between pose_df indices and pel indices
                dist_to_patch_pel_ts_subj = pd.merge_asof(
                    left=dist_to_patch_pel_ts_id_df[subject],
                    right=dist_to_patch_df[dist_to_patch_df["class"] == subject],
                    left_index=True,
                    right_index=True,
                    direction="forward",
                    tolerance=pd.Timedelta("200ms"),
                )
                dist_to_patch_pel_ts_id_df[subject] = dist_to_patch_pel_ts_subj["dist_to_patch"]
            # /ssss>
            # <ssss Get closest subject to patch at each pel del timestep
            patch_df_for_pellets_df["id"] = dist_to_patch_pel_ts_id_df.idxmin(axis=1).values
            # /ssss>
            # <ssss Get closest subject to patch at each wheel timestep
            cum_wheel_dist_subj_df = pd.DataFrame(index=cum_wheel_dist.index, columns=subjects, data=0.0)
            closest_subjects = dist_to_patch_wheel_ts_id_df.idxmin(axis=1)
            wheel_dist = cum_wheel_dist.diff().fillna(cum_wheel_dist.iloc[0])
            # Assign wheel dist to closest subject for each wheel timestep
            for subject in subjects:
                subj_idxs = cum_wheel_dist_subj_df[closest_subjects == subject].index
                cum_wheel_dist_subj_df.loc[subj_idxs, subject] = wheel_dist[subj_idxs]
            cum_wheel_dist_dm[patch] = cum_wheel_dist_subj_df.cumsum(axis=0)
            # /ssss> #/sss> #/ss>
        pellets_stats_df = pd.concat([pellets_stats_df, patch_df_for_pellets_df], ignore_index=True)
        # /s>
    blocks_df.at[block.Index, "patch_info"] = patch_stats_df
    blocks_df.at[block.Index, "pellet_info"] = pellets_stats_df
    blocks_df.at[block.Index, "cum_wheel_dist"] = cum_wheel_dist_dm

Block Iteration:: 0it [00:00, ?it/s]

C:\Users\jai\ProjectAeon\aeon_mecha\aeon\io\reader.py:383: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'BAA-1104048' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  data.loc[data["class"] == i, "class"] = subj
C:\Users\jai\ProjectAeon\aeon_mecha\aeon\analysis\utils.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  distance = distance - distance[0]
C:\Users\jai\AppData\Local\Temp\ipykernel_60492\625617972.py:123: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the c

ValueError: attempt to get argmin of an empty sequence

In [5]:
"""Test individual block."""

block_idx = 272
block = list(blocks_df.itertuples())[block_idx]

# Compute block duration
blocks_df.at[block.Index, "block_duration"] = block.end - block.start
# <s Get subjects within the block:
# Get all unique subjects that visited the environment over the entire exp;
# For each subject, see 'type' of visit most recent to start of block;
# If "Exit", this animal was not in the block.
for arena in arenas:
    if arena in str(block.root):
        cur_env_visits = subject_env_visits[arena]
        break
possible_subjects = cur_env_visits.id.unique().tolist()
subjects = possible_subjects.copy()
for subject in possible_subjects:
    subj_visit_in_time = np.logical_and(
        cur_env_visits.id == subject, cur_env_visits.index < block.start
    )
    if not np.any(subj_visit_in_time):  # if no subject visits prior to block start, drop it
        subjects.remove(subject)
    else:  # if visits, get most recent visit type before block; if "Exit", drop it
        pre_block_visit = cur_env_visits[subj_visit_in_time].iloc[-1]
        # last_visit = cur_env_visits[cur_env_visits.id == subject].iloc[-1]
        if pre_block_visit.type == "Exit":
            subjects.remove(subject)
blocks_df.at[block.Index, "subjects"] = subjects
# /s>
# <s See if we should continue with analyzing this block
cum_pel_ct = 0
for patch in patches:
    r = eval(f"social02.{patch}.DepletionState")
    patch_df = aeon.load(block.root, r, block.start, block.end)
    cum_pel_ct += sum(np.diff(patch_df.index) > pd.Timedelta("1s"))
# if cum_pel_ct < good_block_pel_ct:
#     continue
# /s>
# <s Get pose-tracking info in order to do subject-specific assignments
pose_df = aeon.load(block.root, social02.CameraTop.Pose, block.start, block.end)
pose_df = reader.Pose.class_int2str(pose_df, block.sleap_model_dir)
if len(subjects) == 1:  # fix mistaken sleap assignments for single-subject blocks
    pose_df["class"] = subjects[0]
# /s>
# <s Get per patch data (fill in `patch_info`, `cum_wheel_dist`, `pellet_info` cols of `blocks_df`)
patch_stats_df = pd.DataFrame(index=patches, columns=["mean", "offset"])  # -> patch_info
cum_wheel_dist_dm = DotMap()  # -> cum_wheel_dist
pellets_stats_df = pd.DataFrame(columns=["time", "patch", "threshold", "id"])  # -> pellet_info
for i, patch in enumerate(patches):
    # <ss Get wheel data
    r = eval(f"social02.{patch}.Encoder")
    wheel_df = aeon.load(block.root, r, block.start, block.end)[::50].round(1).astype(np.float32)
    cum_wheel_dist = -distancetravelled(wheel_df.angle)
    # /ss>
    # <ss Get pellets data
    r = eval(f"social02.{patch}.DepletionState")
    patch_df = aeon.load(block.root, r, block.start, block.end)
    rate, offset = patch_df[["rate", "offset"]].iloc[0]
    patch_stats_df.loc[patch, ["mean", "offset"]] = (1 / rate // 100 * 100, offset)
    patch_df_good_indxs = np.concatenate((np.diff(patch_df.index) > pd.Timedelta("1s"), (True,)))
    patch_df_for_pellets_df = patch_df[patch_df_good_indxs].reset_index()[["time", "threshold"]]
    patch_df_for_pellets_df["patch"] = patch
    patch_df_for_pellets_df["id"] = None
    patch_df_for_pellets_df.dropna(subset=["threshold"], inplace=True)
    # drop 1st val as is from block start
    patch_df_for_pellets_df = patch_df_for_pellets_df.iloc[1:].reset_index(drop=True)
    # /ss>
    # <ss Assign data to subjects
    if len(subjects) == 1:
        cum_wheel_dist_dm[patch] = cum_wheel_dist.to_frame(name=subjects[0])
        patch_df_for_pellets_df["id"] = subjects[0]
    else:
        # <sss Assign id based on which subject was closest to patch at time of event
        # <ssss Get distance-to-patch at each pose data timestamp
        patch_xy = np.array(patch_locs[patch][arena]).astype(np.uint32)
        subjects_xy = pose_df[pose_df["part"] == "centroid"][["x", "y"]].values
        dist_to_patch = np.sqrt(np.sum((subjects_xy - patch_xy) ** 2, axis=1))
        dist_to_patch_df = pose_df[["class"]].copy()
        dist_to_patch_df["dist_to_patch"] = dist_to_patch
        # /ssss>
        # <ssss Get distance-to-patch at each wheel ts and pel del ts, organized by subject
        dist_to_patch_wheel_ts_id_df = pd.DataFrame(index=cum_wheel_dist.index, columns=subjects)
        dist_to_patch_pel_ts_id_df = pd.DataFrame(
            index=patch_df_for_pellets_df["time"], columns=subjects
        )
        for subject in subjects:
            # Find closest match between pose_df indices and wheel indices
            dist_to_patch_wheel_ts_subj = pd.merge_asof(
                left=dist_to_patch_wheel_ts_id_df[subject],
                right=dist_to_patch_df[dist_to_patch_df["class"] == subject],
                left_index=True,
                right_index=True,
                direction="forward",
                tolerance=pd.Timedelta("100ms"),
            )
            dist_to_patch_wheel_ts_id_df[subject] = dist_to_patch_wheel_ts_subj["dist_to_patch"]
            # Find closest match between pose_df indices and pel indices
            dist_to_patch_pel_ts_subj = pd.merge_asof(
                left=dist_to_patch_pel_ts_id_df[subject],
                right=dist_to_patch_df[dist_to_patch_df["class"] == subject],
                left_index=True,
                right_index=True,
                direction="forward",
                tolerance=pd.Timedelta("200ms"),
            )
            dist_to_patch_pel_ts_id_df[subject] = dist_to_patch_pel_ts_subj["dist_to_patch"]
        # /ssss>
        # <ssss Get closest subject to patch at each pel del timestamp
        patch_df_for_pellets_df["id"] = dist_to_patch_pel_ts_id_df.idxmin(axis=1).values
        # /ssss>
        # <ssss Get closest subject to patch at each wheel timestamp
        cum_wheel_dist_subj_df = pd.DataFrame(index=cum_wheel_dist.index, columns=subjects, data=0.)
        closest_subjects = dist_to_patch_wheel_ts_id_df.idxmin(axis=1)
        wheel_dist = cum_wheel_dist.diff().fillna(cum_wheel_dist.iloc[0])
        # Assign wheel dist to closest subject for each wheel timestamp
        for subject in subjects:
            subj_idxs = cum_wheel_dist_subj_df[closest_subjects == subject].index
            cum_wheel_dist_subj_df.loc[subj_idxs, subject] = wheel_dist[subj_idxs]
        cum_wheel_dist_dm[patch] = cum_wheel_dist_subj_df.cumsum(axis=0)
        # /ssss> #/sss> #/ss>
    pellets_stats_df = pd.concat([pellets_stats_df, patch_df_for_pellets_df], ignore_index=True)
    # /s>
blocks_df.at[block.Index, "patch_info"] = patch_stats_df
blocks_df.at[block.Index, "pellet_info"] = pellets_stats_df
blocks_df.at[block.Index, "cum_wheel_dist"] = cum_wheel_dist_dm

C:\Users\jai\ProjectAeon\aeon_mecha\aeon\io\reader.py:383: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'BAA-1104045' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  data.loc[data["class"] == i, "class"] = subj
C:\Users\jai\ProjectAeon\aeon_mecha\aeon\analysis\utils.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  distance = distance - distance[0]
C:\Users\jai\AppData\Local\Temp\ipykernel_60492\4086213747.py:114: FutureWarning: The behavior of DataFrame.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  closest_subjects = dist_to_patch_wheel_ts_id_df.idxmin(axis=1)
C:\Users\jai\AppData\Local\Temp\ipykernel_60492\40862

In [6]:
block = list(blocks_df.itertuples())[block_idx]

display(block.patch_info)

display(block.pellet_info)

for patch in patches:
    print(f"Subject cum wheel dist for {patch=}")
    display(block.cum_wheel_dist[patch])

,mean,offset
Patch1,300.0,75.0
Patch2,100.0,75.0
Patch3,500.0,75.0


,time,patch,threshold,id
0,2024-02-13 15:22:02.301983833,Patch1,388.938394,BAA-1104047
1,2024-02-13 15:22:49.179999828,Patch1,145.604313,BAA-1104045
2,2024-02-13 15:23:06.756000042,Patch1,220.218814,BAA-1104045
3,2024-02-13 15:23:25.301983833,Patch1,230.283633,BAA-1104045
4,2024-02-13 15:23:45.619999886,Patch1,294.815140,BAA-1104045
...,...,...,...,...
113,2024-02-13 17:58:58.609983921,Patch2,202.867291,BAA-1104047
114,2024-02-13 17:59:54.309984207,Patch2,90.533233,BAA-1104047
115,2024-02-13 18:00:04.921984196,Patch2,182.347004,BAA-1104047
116,2024-02-13 18:00:30.177984238,Patch2,204.818252,BAA-1104047


Subject cum wheel dist for patch='Patch1'


,BAA-1104045,BAA-1104047
time,,
2024-02-13 15:17:16.001984119,-0.000000,0.000000
2024-02-13 15:17:16.101984024,-0.006136,0.000000
2024-02-13 15:17:16.201983929,0.001534,0.000000
2024-02-13 15:17:16.301983833,0.000000,0.000000
2024-02-13 15:17:16.401984215,-0.003068,0.000000
...,...,...
2024-02-13 18:03:14.501984118,1878.110554,531.869735
2024-02-13 18:03:14.601984024,1878.104418,531.869735
2024-02-13 18:03:14.701983929,1878.110554,531.869735


Subject cum wheel dist for patch='Patch2'


,BAA-1104045,BAA-1104047
time,,
2024-02-13 15:17:16.001984119,0.000000,-0.000000
2024-02-13 15:17:16.101984024,0.000000,0.000000
2024-02-13 15:17:16.201983929,0.000000,0.000000
2024-02-13 15:17:16.301983833,0.000000,-0.007670
2024-02-13 15:17:16.401984215,0.000000,-0.006136
...,...,...
2024-02-13 18:03:14.501984118,6166.591093,12448.501498
2024-02-13 18:03:14.601984024,6166.591093,12448.498430
2024-02-13 18:03:14.701983929,6166.591093,12448.499964


Subject cum wheel dist for patch='Patch3'


,BAA-1104045,BAA-1104047
time,,
2024-02-13 15:17:16.001984119,0.000000,-0.000000
2024-02-13 15:17:16.101984024,0.000000,0.003069
2024-02-13 15:17:16.201983929,0.000000,0.006137
2024-02-13 15:17:16.301983833,0.000000,0.003069
2024-02-13 15:17:16.401984215,0.000000,0.003069
...,...,...
2024-02-13 18:03:14.501984118,408.296965,369.892968
2024-02-13 18:03:14.601984024,408.295431,369.892968
2024-02-13 18:03:14.701983929,408.300033,369.892968


---

In [ ]:
# NOPE

In [ ]:
"""Get strange blocks."""

# Get blocks with no subjects
np.where(blocks_df.subjects.apply(len) == 0)

# Get blocks with < 3 pellets

## Block Plots

In [ ]:
# x,y animal location, over time, per subject

In [ ]:
# 1 / patch_rate, next to boxplots of each pellet threshold per patch

In [ ]:
# Cumulative pellet count over time, per patch, per subject

In [ ]:
# Pellet threshold vals over time, per patch, per subject (1, 0)

In [ ]:
# Cumulative wheel distance over time, per patch, per subject

In [ ]:
# Running cumulative patch preference, per subject: each patch as a line (0, 1)

In [ ]:
# Null distribution 2.5th and 97.5th percentiles with per-patch preference vals, per subject (1, 1)


In [ ]:
# Pairwise Null distribution 2.5th and 97.5th percentiles with per-patch preference vals, per subject (1, 1)

## Overall Plots

In [ ]:
# Weight over time

In [ ]:
# Cumulative patch preference per subject over time per block per patch (by patch number, and patch rate).

# By last 25% of block.

# Analyze (E, E, E; H, H, H) blocks separately

## Research Questions

- In a block, what percentage of time do they end on easy block?

## Usage Questions